In [20]:
import os
import json
import base64
import cv2
import math 
from ultralytics import YOLO
import shutil
import easyocr
import pytesseract
from pytesseract import Output
import re

cv2.destroyAllWindows()

font = cv2.FONT_HERSHEY_SIMPLEX

reader = easyocr.Reader(['es']) 
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract'

def puntoEnContenedor(contenedor,x,y):
    [x1,y1,x2,y2] = contenedor
    return x1 < x < x2 and y1 < y < y2

def cajaEnContenedor(contenedor,contenido):
    [x1,y1,x2,y2] = contenido
    return puntoEnContenedor(contenedor,x1,y1) and puntoEnContenedor(contenedor,x2,y2)

os.environ['YOLO_VERBOSE'] = 'False'

clases = ['Car','Number Plate','Two Wheeler','Auto','Bus','Person','Truck']
cuentaClases = {cls: set() for cls in clases}

model = YOLO('best.pt')

filename = "C0142.MP4"
results = model.track(source=filename, show=False, verbose=False, stream=True)

fotograma = 1
lines = ['fotograma,tipo_objeto,confianza,identificador_tracking,x1,y1,x2,y2,matrícula_en_su_caso,confianza,mx1,my1,mx2,my2,texto_matricula']
video = None
for r in results:
    if video is None:
        video = cv2.VideoWriter('Procesado'+'.'.join(filename.split('.')[:-1])+'.avi', 0, 24, (r.orig_shape[1],r.orig_shape[0]))
    frame = r.plot()
    matriculas = [box for box in r.boxes if int(box.cls.item()) in [1] and box.id is not None]
    persons = [box for box in r.boxes if int(box.cls.item()) in [5] and box.id is not None]
    vehiculos = [box for box in r.boxes if int(box.cls.item()) not in [1,5] and box.id is not None]
    for m in matriculas:
        id = int(m.id.item())
        cuentaClases[clases[int(m.cls.item())]].add(id)
    for v in vehiculos:
        id = int(v.id.item())
        cuentaClases[clases[int(v.cls.item())]].add(id)
        contenedor = v.xyxy.tolist()[0]
        [x1,y1,x2,y2] = contenedor
        mInV = next((m for m in matriculas if cajaEnContenedor(contenedor,m.xyxy.tolist()[0])),None)
        if mInV is not None:
            [mx1,my1,mx2,my2] = [int(v) for v in mInV.xyxy.tolist()[0]]
            cropped = r.orig_img[my1:my2,mx1:mx2,:]
            resized = cv2.resize(cropped,(320,int(cropped.shape[0]/cropped.shape[1]*320)))
            gray = cv2.cvtColor(resized,cv2.COLOR_BGR2GRAY)
            _, img_thres = cv2.threshold(gray,125,255, cv2.THRESH_BINARY_INV)
            mId = int(mInV.id.item())
            mText = re.sub(r'[^0-9A-Z]','',pytesseract.image_to_string(img_thres,config='--psm 11').upper().replace(' ',''))
            mConf = mInV.conf.item()
            lines.append(f"{fotograma},{clases[int(v.cls.item())]},{v.conf.item()},{id},{x1},{y2},{x2},{y2},{mId},{mConf},{mx1},{my2},{mx2},{my2},{mText}")
            if mText != '':
                cv2.putText(frame, f"{mText}", (mx2,my2), font, 0.5, (0,255,0), 2, cv2.LINE_AA)
        else:
            lines.append(f"{fotograma},{clases[int(v.cls.item())]},{v.conf.item()},{id},{x1},{y2},{x2},{y2},,,,,,,,")
    for p in persons:
        id = int(p.id.item())
        cuentaClases[clases[int(p.cls.item())]].add(id)
        contenedor = v.xyxy.tolist()[0]
        [x1,y1,x2,y2] = contenedor
        lines.append(f"{fotograma},{clases[int(p.cls.item())]},{p.conf.item()},{id},{x1},{y2},{x2},{y2},,,,,,,,")
    i = 0
    for cls in clases:
        cv2.putText(frame, f"{cls}: {len(cuentaClases[cls])}", (25,25+i*25), font, 0.5, (0,255,0), 2, cv2.LINE_AA)
        i += 1
    video.write(frame)
    cv2.imshow('Vid',frame)
    cv2.waitKey(1)
    fotograma += 1

cv2.destroyAllWindows()
video.release()
with open('output.csv','w') as fh:
    fh.write("\n".join(lines))

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
